In [1]:
import pandas as pd
import numpy as np

In [2]:
actual = r"D:\Wild deserts photos\concatenated_Dave.xlsx"

In [3]:
df = pd.read_excel(actual, dtype=str)
df['Time'] = pd.to_timedelta(df['Time']) 
df['Date'] = pd.to_datetime(df['Date'])  
df["datetime_converted"] = df["Date"]+df["Time"]

In [4]:
from pathlib import Path
import pandas as pd

root_dir = Path(r"D:\Wild deserts photos\2024_12_WCAM_originals\WCAM01\100RECNX\night_no_fences")

all_dataframes = []

# Recursively search for all .csv files under root_dir
for csv_file in root_dir.rglob("*.csv"):
    # Read the CSV into a DataFrame
    df = pd.read_csv(csv_file)
    
    # Extract information from the parent directory name
    parent_dir = csv_file.parent.name
    
    # Create a new column based on the directory name
    df["model_name"] = parent_dir
    
    # If you want the entire path (relative to root_dir), you could do:
    relative_path = csv_file.relative_to(root_dir)  # e.g., "subfolder/another_subfolder/data.csv"
    # df["relative_path"] = str(relative_path.parent)  # e.g., "subfolder/another_subfolder"

    # If you need multiple levels of folders in separate columns, for example:
    level_one_folder = relative_path.parts[0]  
    df["Location"] = level_one_folder

    # Store the modified DataFrame
    all_dataframes.append(df)

# Concatenate all DataFrames into one (optional)
if all_dataframes:
    master_df = pd.concat(all_dataframes, ignore_index=True)
    print(master_df)
    # Optionally save to a new CSV
    # master_df.to_csv("combined_data.csv", index=False)
else:
    print("No CSV files found.")


                                          path_original   species confidence  \
0     E:\Wild deserts photos\2024_12_WCAM_originals\...  Kangaroo      0.959   
1     E:\Wild deserts photos\2024_12_WCAM_originals\...  Kangaroo      0.953   
2     E:\Wild deserts photos\2024_12_WCAM_originals\...  Kangaroo      0.929   
3     E:\Wild deserts photos\2024_12_WCAM_originals\...  Kangaroo       0.96   
4     E:\Wild deserts photos\2024_12_WCAM_originals\...  Kangaroo      0.958   
...                                                 ...       ...        ...   
2263  E:\Wild deserts photos\2024_12_WCAM_originals\...      none       none   
2264  E:\Wild deserts photos\2024_12_WCAM_originals\...      none       none   
2265  E:\Wild deserts photos\2024_12_WCAM_originals\...      none       none   
2266  E:\Wild deserts photos\2024_12_WCAM_originals\...      none       none   
2267  E:\Wild deserts photos\2024_12_WCAM_originals\...      none       none   

                                       

In [5]:
master_df["datetime_converted"] = pd.to_datetime(master_df["time"], format="%Y:%m:%d %H:%M:%S")
master_df["Date"] = master_df["datetime_converted"].dt.date
master_df["Time"] = master_df["datetime_converted"].dt.time


This code joins the two dfs together to compare. 

In [ ]:

df = df[
    (df["datetime_converted"] > master_df["datetime_converted"].min()) &
    (df["datetime_converted"] < master_df["datetime_converted"].max())
]
joined_df = pd.merge(master_df, df, on = "datetime_converted", how = "right")
cleaned = joined_df[[
    "species", 
    "confidence",
    "Species", 
    "path_original", 
    "datetime_converted", 
    "Location_x", 
    "Location_y", 
    "model_name"
]]
cleaned = cleaned.replace("Red Kangaroo", "Kangaroo")
not_correct = cleaned.loc[~(cleaned["Species"]==cleaned["species"])]
night = not_correct.loc[not_correct["model_name"]=="night_0402"]


In [15]:

master_df = master_df[(master_df["confidence"]!="none")]


In [31]:
master_df['confidence'] = master_df["confidence"].astype(float)

In [32]:
master_df = master_df[(master_df["confidence"]>0.8)]


Extract only images within a 30sec timeframe

In [6]:
images_sorted = master_df.sort_values(by=['datetime_converted'])
selected_images = []
current_anchor = None
records = images_sorted.to_dict(orient="records")  
time_window_sec = 600

In [7]:
for img in records:
    print(img)
    if current_anchor is None:
        # First image in the list becomes our anchor
        current_anchor = img
        continue
    
    time_diff = (img['datetime_converted'] - current_anchor['datetime_converted']).total_seconds()
    
    if time_diff <= time_window_sec:
        # If within the window, pick the higher-confidence image
        if img['confidence'] > current_anchor['confidence']:
            current_anchor = img
    else:
        # Time window exceeded => finalize current_anchor & start a new one
        selected_images.append(current_anchor)
        current_anchor = img
    
    # Don’t forget to append the last anchor if it exists
    

{'path_original': 'E:\\Wild deserts photos\\2024_12_WCAM_originals\\WCAM01\\100RECNX/night_no_fences/labels_images/predict/images/RCNX0001.JPG', 'species': 'Kangaroo', 'confidence': '0.959', 'bbox': '[[1315.7054443359375, 599.194091796875, 810.1574096679688, 776.903076171875]]', 'path_annotated': 'E:\\Wild deserts photos\\2024_12_WCAM_originals\\WCAM01\\100RECNX/night_no_fences/labels_images/predict/annotated/RCNX0001_0.959.JPG', 'label_path': 'E:\\Wild deserts photos\\2024_12_WCAM_originals\\WCAM01\\100RECNX/night_no_fences/labels_images/predict/labels/RCNX0001.txt', 'time': '2024:12:03 23:19:18', 'model_name': 'night_no_fences', 'Location': 'detections_20250225_100643.csv', 'datetime_converted': Timestamp('2024-12-03 23:19:18'), 'Date': datetime.date(2024, 12, 3), 'Time': datetime.time(23, 19, 18)}
{'path_original': 'E:\\Wild deserts photos\\2024_12_WCAM_originals\\WCAM01\\100RECNX/night_no_fences/labels_images/predict/images/RCNX0002.JPG', 'species': 'Kangaroo', 'confidence': '0.953

In [57]:
for img in selected_images:
    print(img['datetime_converted'], img['confidence'], img['species'])

2024-12-03 17:31:06 0.883 Kangaroo
2024-12-03 18:54:27 0.949 Kangaroo
2024-12-03 19:19:10 0.951 Kangaroo
2024-12-03 20:50:59 0.963 Kangaroo
2024-12-03 21:40:51 0.968 Kangaroo
2024-12-03 22:46:25 0.967 Kangaroo
2024-12-03 23:19:18 0.964 Kangaroo
2024-12-03 23:45:46 0.968 Kangaroo
2024-12-04 00:22:08 0.961 Kangaroo
2024-12-04 01:20:55 0.959 Kangaroo
2024-12-04 02:02:56 0.968 Kangaroo
2024-12-04 02:16:16 0.953 Kangaroo
2024-12-04 02:46:57 0.967 Kangaroo
2024-12-04 03:20:45 0.959 Kangaroo
2024-12-04 03:32:55 0.969 Kangaroo
2024-12-04 04:03:46 0.957 Kangaroo
2024-12-04 18:52:04 0.955 Kangaroo
2024-12-04 23:27:34 0.963 Kangaroo
2024-12-05 01:05:39 0.931 Kangaroo
2024-12-05 02:00:18 0.957 Rabbit
2024-12-05 02:46:13 0.924 Rabbit
2024-12-05 03:17:19 0.972 Kangaroo
2024-12-05 04:21:46 0.97 Kangaroo
2024-12-05 05:02:16 0.916 Kangaroo
2024-12-05 20:24:57 0.969 Kangaroo
2024-12-05 20:37:16 0.97 Kangaroo
2024-12-05 22:25:07 0.972 Kangaroo
2024-12-05 22:45:17 0.969 Cat
2024-12-05 23:52:31 0.965 Kanga

In [8]:
selected_images_df = pd.DataFrame(selected_images)

In [63]:

df = df[
    (df["datetime_converted"] > master_df["datetime_converted"].min()) &
    (df["datetime_converted"] < master_df["datetime_converted"].max())
]
joined_df = pd.merge(selected_images_df, df, on = "datetime_converted", how = "outer")
cleaned = joined_df[[
    "species", 
    "confidence",
    "Species", 
    "path_original", 
    "datetime_converted", 
    "Location_x", 
    "Location_y", 
    "model_name"
]]
cleaned = cleaned.replace("Red Kangaroo", "Kangaroo")
not_correct = cleaned.loc[~(cleaned["Species"]==cleaned["species"])]
night = not_correct.loc[not_correct["model_name"]=="night_0402"]


In [68]:
test = pd.merge_asof(selected_images_df, df.sort_values(by=['datetime_converted']), on = "datetime_converted", direction='nearest')

In [70]:
cleaned_test = test[[
    "species", 
    "confidence",
    "Species", 
    "path_original", 
    "datetime_converted", 
    "Location_x", 
    "Location_y", 
    "model_name",
    "datetime_converted_x",
    "datetime_converted_y"
]]

KeyError: "['datetime_converted_x', 'datetime_converted_y'] not in index"

In [71]:
selected_images_df.to_csv("december_ai.csv")